### Scrapping MEO Forum

In [1]:
import requests
import re
import json
import logging
import pandas
from collections import OrderedDict
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import logging
from collections import OrderedDict
from tqdm import tqdm_notebook as tqdm
from random import seed
from random import randint
from datetime import datetime
from webdriver_manager.chrome import ChromeDriverManager
from time import time       # Other imports
from time import sleep
import csv
import time
from tqdm import tqdm_notebook as tqdm
from random import seed
from random import randint
from datetime import datetime
from selenium import webdriver
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
from time import sleep


### Drivers

In [2]:
driver = webdriver.Chrome("C:/Users/GSantos/Documents/GABRIEL RAVI/Web Scrapping/chromedriver_win32_2/chromedriver.exe")

In [3]:
driver.get("https://forum.meo.pt/search/activity/recent")

In [4]:
content = driver.page_source
soup = BeautifulSoup(content)

In [5]:
base_url = "https://forum.meo.pt/"

### Scroll Down

In [6]:
from time import sleep
def scroll_down(driver) -> None:
        """Scrolling down the page and taking the URLs"""
        global listona
        global posts_url
        
        listona = []
        listona2 = []
        posts_url = []
        last_height = 0
        
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(3)
            
            
            elements = driver.find_elements_by_class_name('topic-view_link')
            for elem in elements:
                urls = elem.get_attribute('href')
                listona.append(urls)    
                
                
                
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if new_height == last_height:
                break
            last_height = new_height
            
            posts_url += listona

In [7]:
scroll_down(driver)

In [8]:
posts_url

['https://forum.meo.pt/internet-fixa-e-movel-11/esclarecimento-nas-compras-com-meos-147983',
 'https://forum.meo.pt/tv-e-pacotes-9/reclamacao-falta-de-profissionalismo-e-burla-147992',
 'https://forum.meo.pt/outros-servicos-e-apps-meo-14/entrega-de-equipamentos-147991',
 'https://forum.meo.pt/telemovel-12/facturacao-indevida-147990',
 'https://forum.meo.pt/internet-fixa-e-movel-11/internet-1gb-400mbps-falso-147436',
 'https://forum.meo.pt/telefone-fixo-13/como-parar-com-as-vendas-da-meo-ao-telefone-147984',
 'https://forum.meo.pt/telemovel-12/compras-com-meos-147989',
 'https://forum.meo.pt/telemovel-12/pagamento-da-prestacao-147985',
 'https://forum.meo.pt/tv-e-pacotes-9/instalacao-inacabado-147988',
 'https://forum.meo.pt/internet-fixa-e-movel-11/cobertura-meo-fibra-147987',
 'https://forum.meo.pt/telemovel-12/volte-na-meo-145887',
 'https://forum.meo.pt/telemovel-12/contactos-aparecem-como-numero-desconhecido-no-iphone-147986',
 'https://forum.meo.pt/tv-e-pacotes-9/instalacao-de-fib

### Coletando Informações do Post

In [47]:
# Exemplo com 3 comentários
driver.get("https://forum.meo.pt/internet-fixa-e-movel-11/quer-melhorar-a-cobertura-de-wifi-em-sua-casa-147834")

In [69]:
def info_post():
    
    global type_post_principal
    global title_post_principal
    global user_post_principal
    global class_user_principal
    global data_post_principal
    global texto_post_principal
    global likes_post_principal

    
    ### Category of the post
    try:
        type_post_principal = driver.find_element_by_xpath('//html/body/main/div[3]/div/ul/li[3]/a').text
    except:
        pass
    if type_post_principal == ['']:
        type_post_principal = ["NaN"]   
    
    ### Title of the post
    try:
        title_post_principal = driver.find_element_by_css_selector('body > main > div.full-width.Template-content > div > div.col.col--main.has--side.qa-div-main > div > div > div.box.qa-topic-first-post > header > h1').text
    except:
        pass
    if title_post_principal == ['']:
        title_post_principal = ["NaN"]   
    
    ### User of the post
    try:
        user_post_principal = driver.find_element_by_xpath('/html/body/main/div[5]/div/div[1]/div/div/div[1]/div[2]/div[2]/ul/li[1]/div/a').text
    except:
        pass
    if user_post_principal == ['']:
        user_post_principal = ["NaN"]   
    
    ### Class of the Userthat posted
    try:
        class_user_principal = driver.find_element_by_css_selector('.usertitle.qa-usertitle').text
    except:
        pass
    if class_user_principal == ['']:
        class_user_principal = ["NaN"]   
    
    ### Data Post
    try:
        data_post_principal = driver.find_element_by_css_selector('.tooltip-trigger.js-tooltip-trigger.qa-latest-post-time').get_attribute('datetime')
    except:
        pass
    if data_post_principal == ['']:
        data_post_principal = ["NaN"]   
    
    ### Post itself
    try:
        texto_post_principal = driver.find_element_by_xpath('/html/body/main/div[5]/div/div[1]/div/div/div[1]/div[2]/div[3]').text.replace("\n"," ")
    except:
        pass
    if texto_post_principal == ['']:
        texto_post_principal = ["NaN"]   
    
    ### Likes of the post
    try:
        likes_post_principal = driver.find_element_by_xpath('/html/body/main/div[5]/div/div[1]/div/div/div[1]/div[3]/div[4]/div[2]/div/div[2]/div/span').text
    except:
        pass
    if likes_post_principal == ['']:
        likes_post_principal = ["NaN"]   
    

In [70]:
info_post()
type_post_principal
title_post_principal
user_post_principal
class_user_principal
data_post_principal

texto_post_principal

likes_post_principal

'5 likes'

### Collecting all the answers 

In [73]:
def scrap_comments():
    
    global user_posts
    global users_post2
    global class_users
    global class_users2
    global data_posts
    global data_posts2
    global texto_posts
    global texto_posts2
    global likes_posts
    global likes_post2
    
    user_posts = []
    users_post2 = []
    class_users = []
    class_users2 = []
    data_posts = []
    data_posts2 = []
    texto_posts = []
    texto_posts2 = []
    likes_post = []
    likes_post2 = []
    
    u_post = driver.find_elements_by_class_name('group--inline')
    cu_post = driver.find_elements_by_css_selector('.usertitle.qa-usertitle')
    post_date = driver.find_elements_by_xpath("//a[@class='post__date']/div/time")
    texto_post = driver.find_elements_by_css_selector('.post__content.js-content--original.qa-topic-post-content.post__content--new-editor')
    likes_u_post = driver.find_elements_by_css_selector('.tooltip-trigger.js-tooltip-trigger.link--quiet.qa-total-likes-count')

    
    for i in u_post:
        user_posts.append(i.text)
    
    for i in cu_post:
        class_users.append(i.text)
    
    for i in post_date:
        data_posts.append(i.get_attribute('datetime'))
    data_posts.insert(0, data_post_principal) 
    for i in texto_post:
        texto_posts.append(i.text.replace("\n"," "))
   
    for i in likes_u_post:
        likes_post.append(i.text)#.replace("likes"," ")
    
    
    users_post2 += user_posts
    class_users2 += class_users
    data_posts2 += data_posts
    texto_posts2 += texto_posts
    likes_post2 += likes_post

In [75]:
user_posts = []
users_post2 = []

In [76]:
scrap_comments()
# print(len(users_post2))
# print(len(class_users2))
# print(len(data_posts2))
# print(len(texto_posts2))
likes_post2

['5 likes',
 '2 likes',
 '2 likes',
 '2 likes',
 '',
 '1 like',
 '2 likes',
 '',
 '2 likes',
 '2 likes',
 '2 likes',
 '4 likes',
 '4 likes',
 '4 likes',
 '2 likes',
 '',
 '1 like',
 '',
 '',
 '',
 '',
 '1 like',
 '',
 '',
 '1 like',
 '1 like']

In [77]:
texto_posts2

['Agora também é possível ver a TV do MEO na nova Box Android TV. Com uma experiência personalizada e uma navegação inovadora através do novo comando MEO, a TV do MEO está como nunca a viu. Agora já é possível ver os seus programas, ter acesso aos canais premium, gravações automáticas e videoclube da sua TV na Android TV.  Veja as funcionalidades que tem disponíveis?  Menu do MEO na Android TV. Um menu que lhe dá acesso direto às principais categorias: “Para Si”: acesso direto aos canais mais vistos em sua casa, à grelha de todos os canais, aos conteúdos que estão a passar na TV organizados por temáticas, às “Gravações” ou filmes do Videoclube que deixou a meio e a todo um mundo de sugestões do MEO com as últimas novidades da TV. “Gravações”: acesso a todos os conteúdos que passaram na TV nos últimos 7 dias organizados de diferentes formas: acesso direto aos últimos canais a que acedeu nas “Gravações”, ao “Continuar a ver” os programas que deixou a meio, aos conteúdos das “Gravações” o

### Function to add the data

In [78]:
from time import time

In [84]:
start_time = time()

post_data = OrderedDict()

post_data['Tempo_recolha'] = []
post_data['Categoria'] = []
post_data['Empresa'] = []
post_data['n'] = []
post_data['URL_Post'] = []
post_data['Texto_Pub'] = []
post_data['Theme_Pub'] = []
post_data['User_Pub'] = []
post_data['Class_User_Pub'] = []
post_data['Data_Pub'] = []
post_data['Texto_Pub'] = []
post_data['Likes_Pub'] = []

post_data['User_Conv'] = []
post_data['Class_Conv'] = []
post_data['Data_Conv'] = []
post_data['Textos_Conv'] = []
post_data['Likes_Conv'] = []

post_data['N_respostas'] = []

#post_data['Autor_Post_ID'] = []

## Create the List 

In [85]:
base_url1 = 'https://forum.meo.pt/search/activity/recent/index'
base_url2 = '.html'
empresa= 'MEO'
N = 20
categoria = "Telecomunicação"

lista = pd.DataFrame({ 'N' : range(1, N + 1 ,1),
                      'Categoria': categoria,
                     "URL_Post1": base_url1,
                     "URL_Post2": base_url2,
                     "Empresa": empresa})


lista["posts"] = lista["URL_Post1"] + lista["N"].astype(str) + lista["URL_Post2"] 
lista = lista.drop(columns=['URL_Post1', 'URL_Post2'])
lista

,N,Categoria,Empresa,posts
0,1,Telecomunicação,MEO,https://forum.meo.pt/search/activity/recent/in...
1,2,Telecomunicação,MEO,https://forum.meo.pt/search/activity/recent/in...
2,3,Telecomunicação,MEO,https://forum.meo.pt/search/activity/recent/in...
3,4,Telecomunicação,MEO,https://forum.meo.pt/search/activity/recent/in...
4,5,Telecomunicação,MEO,https://forum.meo.pt/search/activity/recent/in...
5,6,Telecomunicação,MEO,https://forum.meo.pt/search/activity/recent/in...
6,7,Telecomunicação,MEO,https://forum.meo.pt/search/activity/recent/in...
7,8,Telecomunicação,MEO,https://forum.meo.pt/search/activity/recent/in...
8,9,Telecomunicação,MEO,https://forum.meo.pt/search/activity/recent/in...
9,10,Telecomunicação,MEO,https://forum.meo.pt/search/activity/recent/in...


In [86]:
# lista["posts"][3]
lista["posts"][10]

'https://forum.meo.pt/search/activity/recent/index11.html'

## Here comes the Magic

In [87]:
from tqdm.notebook import tqdm

In [88]:
posts_url = []

In [89]:
if __name__ == "__main__":
    
    size_ = 0
    with tqdm(total=len(lista["posts"][:1])) as pbar:
        for company, lista_posts in enumerate(lista["posts"][:1], start=0):
            
            pbar.set_description(f'Tamanho da base: {size_} obs')
            pbar.update(1)
            
            driver.get(lista_posts)
            
            scroll_down(driver)
            
            page_source = driver.page_source
            profile_bs = BeautifulSoup(page_source, 'lxml')

            company_ = lista["Empresa"][company]
            
            with tqdm(total=len(posts_url)) as pbar2:
                    for n, i in enumerate(posts_url, start=1):
                        pbar2.update(1)
                        pbar2.set_description(f'Posts da {company_}')
                                                
                        driver.get(i)
                        
                        info_post()                   
                                                
                        driver.get(i)
                        
                        user_post = []
                        class_users = []
                        data_posts = []
                        texto_posts = []
                        likes_post = []
                        
                        comment_url2 = []
                        user_post2 = []
                        class_users2 = []
                        data_posts2 = []
                        texto_posts2 = []
                        likes_post2 = []
                        
                        scrap_comments()
                                                
                           
                        while len(driver.find_elements_by_xpath("//*[contains(@aria-label,'Load more comments')]")) == 1:

                            page_source = driver.page_source
                            profile_bs = BeautifulSoup(page_source, 'lxml')

                            comment_url_aux = []

                            elements = driver.find_elements_by_css_selector('.page-link.lia-link-navigation.lia-custom-event')


                            for elem in elements:
                                urls = elem.get_attribute('href')
                                comment_url_aux.append(urls)  


                            if comment_url_aux == []:
                                break

    #                         driver.get(comment_url_aux)

    #                         scrap_comments()

                            if len(texto_posts2) == 0:
                                    users_post2, class_users2, data_posts2, texto_posts2,likes_post2 = ["NaN"], ["NaN"], ["NaN"], ["NaN"], ["NaN"]


                        comment_comment = comment_url2
                        if len(comment_url2) > 0:
                            for comment_count, aux in enumerate(comment_comment,start=0):
                                comment_url2 = []
                                driver.get(comment_comment[comment_count])
                                scrap_comments()

                                    
                        post_data['Tempo_recolha'] = np.concatenate((post_data['Tempo_recolha'], np.tile(datetime.now(), len(texto_posts2))))
                        post_data['Categoria'] = np.concatenate((post_data['Categoria'], np.tile(lista['Categoria'][company], len(texto_posts2))))
                        post_data['Empresa'] = np.concatenate((post_data['Empresa'], np.tile(lista['Empresa'][company], len(texto_posts2))))
       #                 post_data['post_author_url'] = np.concatenate((post_data['post_author_url'], np.tile(lista['URL_Post'][company], len(texto_posts2))))
                        post_data['n'] = np.concatenate((post_data['n'], np.tile(n, len(texto_posts2))))
                        post_data['URL_Post'] = np.concatenate((post_data['URL_Post'], np.tile(i, len(texto_posts2))))

                        post_data['Texto_Pub'] = np.concatenate((post_data['Texto_Pub'], np.tile(texto_post_principal, len(texto_posts2))))
                        post_data['Theme_Pub'] = np.concatenate((post_data['Theme_Pub'], np.tile(title_post_principal, len(texto_posts2))))
                        post_data['User_Pub'] = np.concatenate((post_data['User_Pub'], np.tile(user_post_principal, len(texto_posts2))))
                        post_data['Class_User_Pub'] = np.concatenate((post_data['Class_User_Pub'], np.tile(class_user_principal, len(texto_posts2))))
                        post_data['Data_Pub'] = np.concatenate((post_data['Data_Pub'], np.tile(data_post_principal, len(texto_posts2))))
                        post_data['Likes_Pub'] = np.concatenate((post_data['Likes_Pub'], np.tile(likes_post_principal, len(texto_posts2))))
                    
                    
                        size_ = len(post_data['URL_Post'])

                        post_data['User_Conv'] = post_data['User_Conv'] + users_post2
                        post_data['Class_Conv'] = post_data['Class_Conv'] + class_users2
                        post_data['Data_Conv'] = post_data['Data_Conv'] + data_posts2
                        post_data['Textos_Conv'] = post_data['Textos_Conv'] + texto_posts2
                        post_data['Likes_Conv'] = post_data['Likes_Conv'] + likes_post2
    
    

In [64]:
post_data['Texto_Pub']

array(['Boas Temos 1 contrato com período de fidelização a decorrer, com condições promocionais ativas, até 01 jan 2022, tem dois telemóveis, no pacote e 1000 (mil) MEOS. Alguém pode esclarecer o que acontecerá se comprar Extensor MEO Smart Wi Fi, no valor de €40 com 350 MEOS ou outros produtos? Vão fidelizar por mais 18 meses com ou sem promoções existentes? Abril 2021+18 meses =  outubro 2023? Não queremos perder as promoções, no pacote M4O. Atenciosamente José',
       'Boas Temos 1 contrato com período de fidelização a decorrer, com condições promocionais ativas, até 01 jan 2022, tem dois telemóveis, no pacote e 1000 (mil) MEOS. Alguém pode esclarecer o que acontecerá se comprar Extensor MEO Smart Wi Fi, no valor de €40 com 350 MEOS ou outros produtos? Vão fidelizar por mais 18 meses com ou sem promoções existentes? Abril 2021+18 meses =  outubro 2023? Não queremos perder as promoções, no pacote M4O. Atenciosamente José',
       'Boas Temos 1 contrato com período de fidelização a d

In [90]:
post_data2 = {'Data_recolha':post_data['Tempo_recolha'],
              'Categoria':post_data['Categoria'],
              'Empresa':post_data['Empresa'],
              'n': post_data['n'],
              'URL_Post': post_data['URL_Post'],
              'Title_Post': post_data['Theme_Pub'],
              'User_Post': post_data['User_Pub'],
              'Class_User_Post': post_data['Class_User_Pub'],
              'Texto_Post' : post_data['Texto_Pub'],
              'Data_Post' : post_data['Data_Pub'] ,
              'Likes_Post':post_data['Likes_Pub'], 

              'Users' : post_data['User_Conv'],
              'Class_Users': post_data['Class_Conv'], 
              'Data_coments': post_data['Data_Conv'],
              'Coments': post_data['Textos_Conv'], 
              'Likes_Coments': post_data['Likes_Conv']
             }

post_data3 = pd.DataFrame.from_dict(post_data2, orient='index')
base = post_data3.transpose()
base2 = base[base['Texto_Post'].notna()]
base2

,Data_recolha,Categoria,Empresa,n,URL_Post,Title_Post,User_Post,Class_User_Post,Texto_Post,Data_Post,Likes_Post,Users,Class_Users,Data_coments,Coments,Likes_Coments
0,2021-03-25 18:28:47.239150,Telecomunicação,MEO,1,https://forum.meo.pt/internet-fixa-e-movel-11/...,Esclarecimento nas compras com MEOS,A R R,Dinamizador Júnior,Boas Temos 1 contrato com período de fidelizaç...,2021-03-25,5 likes,A R R,Dinamizador Júnior,2021-03-25,Boas Temos 1 contrato com período de fidelizaç...,
1,2021-03-25 18:28:47.239150,Telecomunicação,MEO,1,https://forum.meo.pt/internet-fixa-e-movel-11/...,Esclarecimento nas compras com MEOS,A R R,Dinamizador Júnior,Boas Temos 1 contrato com período de fidelizaç...,2021-03-25,5 likes,Ricardo27,Super User,2021-03-25,Penso que esse artigo não obriga a fidelização...,1 like
2,2021-03-25 18:28:47.239150,Telecomunicação,MEO,1,https://forum.meo.pt/internet-fixa-e-movel-11/...,Esclarecimento nas compras com MEOS,A R R,Dinamizador Júnior,Boas Temos 1 contrato com período de fidelizaç...,2021-03-25,5 likes,A R R,Dinamizador Júnior,2021-03-25,Ricardo27 Obrigado pela informação. Um familia...,
3,2021-03-25 18:28:51.294177,Telecomunicação,MEO,2,https://forum.meo.pt/tv-e-pacotes-9/reclamacao...,"Reclamação, falta de profissionalismo e burla",Alicia sousa e Nuno Soares,Novo Membro,Em março comprei uma habitação no conselho de ...,2021-03-25,5 likes,Alicia sousa e Nuno Soares,Novo Membro,2021-03-25,Em março comprei uma habitação no conselho de ...,
4,2021-03-25 18:28:51.294177,Telecomunicação,MEO,2,https://forum.meo.pt/tv-e-pacotes-9/reclamacao...,"Reclamação, falta de profissionalismo e burla",Alicia sousa e Nuno Soares,Novo Membro,Em março comprei uma habitação no conselho de ...,2021-03-25,5 likes,Alicia sousa e Nuno Soares,Novo Membro,2021-03-25,"Na altura que este texto foi escrito, a meo li...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,2021-03-25 18:39:57.335945,Telecomunicação,MEO,147,https://forum.meo.pt/temas-livres-21/mensalida...,Mensalidade superior ao contratualizado em 4 e...,jcarlos1965,Novo Membro,Dirigi-me a uma loja oficial da MEO para me de...,2021-03-24,1 like,jcarlos1965,None,2021-03-24,Dirigi-me a uma loja oficial da MEO para me de...,
902,2021-03-25 18:39:57.335945,Telecomunicação,MEO,147,https://forum.meo.pt/temas-livres-21/mensalida...,Mensalidade superior ao contratualizado em 4 e...,jcarlos1965,Novo Membro,Dirigi-me a uma loja oficial da MEO para me de...,2021-03-24,1 like,Ricardo27,None,2021-03-24,Veja o período que está a ser faturado. Não se...,
903,2021-03-25 18:40:02.836767,Telecomunicação,MEO,148,https://forum.meo.pt/tv-e-pacotes-9/cancelamen...,cancelamento,AndreBizarro,Novo Membro,Boa tarde. Apenas tenho serviço de Satelite co...,2021-03-24,1 like,AndreBizarro,None,2021-03-24,Boa tarde. Apenas tenho serviço de Satelite co...,
904,2021-03-25 18:40:07.857083,Telecomunicação,MEO,149,https://forum.meo.pt/internet-fixa-e-movel-11/...,Desativar DHCPv6 no router GR241AG,Meo2121,Novo Membro,Foi instalado um router GR241AG da MEO na empr...,2021-03-24,1 like,Meo2121,None,2021-03-24,Foi instalado um router GR241AG da MEO na empr...,


In [91]:
base

,Data_recolha,Categoria,Empresa,n,URL_Post,Title_Post,User_Post,Class_User_Post,Texto_Post,Data_Post,Likes_Post,Users,Class_Users,Data_coments,Coments,Likes_Coments
0,2021-03-25 18:28:47.239150,Telecomunicação,MEO,1,https://forum.meo.pt/internet-fixa-e-movel-11/...,Esclarecimento nas compras com MEOS,A R R,Dinamizador Júnior,Boas Temos 1 contrato com período de fidelizaç...,2021-03-25,5 likes,A R R,Dinamizador Júnior,2021-03-25,Boas Temos 1 contrato com período de fidelizaç...,
1,2021-03-25 18:28:47.239150,Telecomunicação,MEO,1,https://forum.meo.pt/internet-fixa-e-movel-11/...,Esclarecimento nas compras com MEOS,A R R,Dinamizador Júnior,Boas Temos 1 contrato com período de fidelizaç...,2021-03-25,5 likes,Ricardo27,Super User,2021-03-25,Penso que esse artigo não obriga a fidelização...,1 like
2,2021-03-25 18:28:47.239150,Telecomunicação,MEO,1,https://forum.meo.pt/internet-fixa-e-movel-11/...,Esclarecimento nas compras com MEOS,A R R,Dinamizador Júnior,Boas Temos 1 contrato com período de fidelizaç...,2021-03-25,5 likes,A R R,Dinamizador Júnior,2021-03-25,Ricardo27 Obrigado pela informação. Um familia...,
3,2021-03-25 18:28:51.294177,Telecomunicação,MEO,2,https://forum.meo.pt/tv-e-pacotes-9/reclamacao...,"Reclamação, falta de profissionalismo e burla",Alicia sousa e Nuno Soares,Novo Membro,Em março comprei uma habitação no conselho de ...,2021-03-25,5 likes,Alicia sousa e Nuno Soares,Novo Membro,2021-03-25,Em março comprei uma habitação no conselho de ...,
4,2021-03-25 18:28:51.294177,Telecomunicação,MEO,2,https://forum.meo.pt/tv-e-pacotes-9/reclamacao...,"Reclamação, falta de profissionalismo e burla",Alicia sousa e Nuno Soares,Novo Membro,Em março comprei uma habitação no conselho de ...,2021-03-25,5 likes,Alicia sousa e Nuno Soares,Novo Membro,2021-03-25,"Na altura que este texto foi escrito, a meo li...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,2021-03-25 18:39:57.335945,Telecomunicação,MEO,147,https://forum.meo.pt/temas-livres-21/mensalida...,Mensalidade superior ao contratualizado em 4 e...,jcarlos1965,Novo Membro,Dirigi-me a uma loja oficial da MEO para me de...,2021-03-24,1 like,jcarlos1965,None,2021-03-24,Dirigi-me a uma loja oficial da MEO para me de...,
902,2021-03-25 18:39:57.335945,Telecomunicação,MEO,147,https://forum.meo.pt/temas-livres-21/mensalida...,Mensalidade superior ao contratualizado em 4 e...,jcarlos1965,Novo Membro,Dirigi-me a uma loja oficial da MEO para me de...,2021-03-24,1 like,Ricardo27,None,2021-03-24,Veja o período que está a ser faturado. Não se...,
903,2021-03-25 18:40:02.836767,Telecomunicação,MEO,148,https://forum.meo.pt/tv-e-pacotes-9/cancelamen...,cancelamento,AndreBizarro,Novo Membro,Boa tarde. Apenas tenho serviço de Satelite co...,2021-03-24,1 like,AndreBizarro,None,2021-03-24,Boa tarde. Apenas tenho serviço de Satelite co...,
904,2021-03-25 18:40:07.857083,Telecomunicação,MEO,149,https://forum.meo.pt/internet-fixa-e-movel-11/...,Desativar DHCPv6 no router GR241AG,Meo2121,Novo Membro,Foi instalado um router GR241AG da MEO na empr...,2021-03-24,1 like,Meo2121,None,2021-03-24,Foi instalado um router GR241AG da MEO na empr...,


In [92]:
base.URL_Post[0]

'https://forum.meo.pt/internet-fixa-e-movel-11/esclarecimento-nas-compras-com-meos-147983'

### Export

In [93]:
writer = pd.ExcelWriter('BASES/MEO_v2.xlsx',
                        engine='xlsxwriter',options={'strings_to_urls': False}) #.format(staName2)
base.to_excel(writer)
writer.close()